## Evaluation: RAG

This is notebook for **RAG model evaluation** for estimating improvements over the base model and compare with other adapted models. We choose to evaluate on the set of benchmarks from [Open Medical-LLM Leaderboard](https://huggingface.co/spaces/openlifescienceai/open_medical_llm_leaderboard) including:

* [MedMCQA](https://huggingface.co/datasets/openlifescienceai/medmcqa) - MCQ, 200 samples from validation split
* [MedQA](https://huggingface.co/datasets/GBaker/MedQA-USMLE-4-options-hf) - MCQ, 200 samples from validation split
* [MMLU](https://huggingface.co/datasets/cais/mmlu) - MCQ, 200 samples from test splits of 6 medical subsets
* [PubMedQA](https://huggingface.co/datasets/qiaojin/PubMedQA) - QA, 200 samples from train split of pqa_labeled subset

### Setup

In [1]:
%%capture
!pip install datasets vllm
!pip install langchain_chroma langchain_huggingface chromadb numpy

In [2]:
import re
from tqdm import tqdm
import math
import pandas as pd
from datasets import load_dataset, concatenate_datasets
from vllm import LLM, SamplingParams
import torch

INFO 04-14 02:59:59 [__init__.py:239] Automatically detected platform cuda.


2025-04-14 03:00:03.164512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744599603.638172      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744599603.756290      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### RAG 

In [ ]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import os
from typing import List, Any
from datasets import load_dataset
import time


class DB:
    def __init__(self,
                 collection_name: str = "medical_rag",
                 embedding_model_name: str = "multi-qa-MiniLM-L6-cos-v1") -> None:
        """
        Initialize the DB instance.

        :param db_path: Directory path where the database is stored or will be created.
        :param embedding_model_name: Name of the embedding model used for generating embeddings.
        """
        self.collection_name = collection_name
        self.db_path = os.path.join(os.getcwd(), collection_name)
        self.embedding_model_name = embedding_model_name
        self.db = None

        model_kwargs = {'device': 'cuda'}
        encode_kwargs = {'normalize_embeddings': True}
        self.embedder = HuggingFaceEmbeddings(
            model_name=embedding_model_name,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs,
        )

        if os.path.exists(self.db_path):
            print("DB exists, loading it...")
            self.db = Chroma(
                collection_name=collection_name,
                embedding_function=self.embedder,
                persist_directory=self.db_path,
            )
            print("DB loaded.")
        else:
            print("DB does not exist, creating it...")
            os.makedirs(self.db_path, exist_ok=True)
            start_time = time.time()
            self._populate_db()
            end_time = time.time()
            print("DB populated.")
            print(f"Time taken to populate DB: {end_time - start_time} sec")

    def _populate_db(self) -> None:
        """
        Populate the database from the dataset.
        Loads the 'MedRAG/textbooks' and 'MedRAG/statpearls' dataset and creates a new Chroma database from the 'contents' field.
        """
        ds_textbooks = load_dataset("MedRAG/textbooks")
        ds_statpearls = load_dataset("MilyaShams/MedRAG_statpearls")

        contents = ds_textbooks["train"]["contents"]
        contents.extend(ds_statpearls["train"]["contents"])

        self.db = Chroma.from_texts(
            texts=contents,
            embedding=self.embedder,
            persist_directory=self.db_path,
            collection_name=self.collection_name,
        )

    def query(self, queries: List[str], top_k: int = 3) -> List[List[str]]:
        """
        Query the database for the top-k most relevant chunks for a batch of queries.

        :param queries: A list of user search queries.
        :param top_k: The number of top relevant results to retrieve for each query.
        :return: A list where each element is a list of retrieved document contents for the corresponding query.
        """
        if not self.db:
            raise ValueError("Database is not initialized.")

        batch_results = []
        print(f"Querying DB for {len(queries)} queries...")

        for query in queries:
            try:
                results = self.db.similarity_search(query, k=top_k)
                retrieved_docs = [doc.page_content for doc in results]
                batch_results.append(retrieved_docs)
            except Exception as e:
                print(f"Error during similarity search for query '{query[:50]}...': {e}")
                batch_results.append([])
                
        return batch_results

    def close(self) -> None:
        """
        Close the database connection.
        """
        if self.db:
            self.db.close()

In [ ]:
from vllm import LLM, SamplingParams
from typing import List, Optional
import torch


class LLMInference:
    """
    A class to handle text generation using the vLLM inference engine.
    """

    def __init__(self,
                 model_name: str,
                #  dtype: str='auto',
                 dtype=torch.float16,
                 trust_remote_code: bool=True,
                 quantization: Optional[str]=None,
                 tensor_parallel_size=2,
        ):
        """
        Initializes the VLLMInference object with the specified parameters and loads the model.

        Args:
            model_name (str): The name or path of the model to be loaded.
            dtype (str, optional): The data type to use for the model. Defaults to 'auto'.
            trust_remote_code (bool, optional): Whether to trust remote code when loading the model. Defaults to True.
            quantization (Optional[str], optional): The quantization mode to use for the model. Defaults to None.
        """
        self.model_name = model_name
        self.dtype = dtype
        self.trust_remote_code = trust_remote_code
        self.quantization = quantization
        self.seed = 4242

        self.llm = LLM(
            model=self.model_name,
            dtype=self.dtype,
            trust_remote_code=self.trust_remote_code,
            quantization=self.quantization,
            seed=self.seed,
            tensor_parallel_size=tensor_parallel_size,
        )

    def generate(self,
                 prompts: List[str],
                 max_tokens: int=4096,
                 temperature: float=0.01,
                 top_p: float=1.0,
                 top_k: int=-1,
                 **kwargs
        ) -> List[str]:
        """
        Generates text based on the provided prompts.

        Args:
            prompts (List[str]): A list of input prompts for text generation.
            temperature (float, optional): Sampling temperature to use. Defaults to 1.0.
            top_p (float, optional): Nucleus sampling probability. Defaults to 1.0.
            max_tokens (int, optional): Maximum number of tokens to generate. Defaults to 4096.

        Returns:
            List[str]: A list of generated texts corresponding to each prompt.
        """
        sampling_params = SamplingParams(
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            max_tokens=max_tokens,
            seed=self.seed,
            **kwargs
        )

        outputs = self.llm.generate(prompts, sampling_params)
        return [output.outputs[0].text for output in outputs]

In [ ]:
class RAG:
    """
    A Retrieval-Augmented Generation (RAG) class that combines a document database
    with an LLM inference engine. It retrieves context using the DB class and uses the
    LLM to generate a response based on the query and retrieved documents.
    """

    def __init__(self,
                 collection_name: str = "med_textbooks",
                 embedding_model_name: str = "multi-qa-MiniLM-L6-cos-v1",
                 llm_name: str="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B") -> None:
        """
        Initialize the RAG instance.

        :param llm_name: The HuggingFace model name or path.
        """
        self.db = DB(collection_name=collection_name,
                     embedding_model_name=embedding_model_name)
        self.llm = LLMInference(model_name=llm_name)
        
    def get_response(self, queries: List[str], top_k: int = 3) -> List[str]:
        """
        Retrieve relevant documents, construct prompts, and generate responses for a batch of queries.

        :param queries: A list of user queries.
        :param top_k: Number of top relevant documents to retrieve for each query.
        :return: A list of generated response texts corresponding to each input query.
        """
        if not queries:
            return []

        print(f"Processing batch of {len(queries)} queries...")
        batch_retrieved_docs = self.db.query(queries, top_k=top_k)

        final_prompts = []
        for i, query in enumerate(queries):
            retrieved_docs = batch_retrieved_docs[i]
            if retrieved_docs:
                context = "\n\n".join(retrieved_docs)
                prompt = f"""Based on the following retrieved documents, answer the user's query. Filter out irrelevant information and synthesize the answer.

Retrieved documents:
---
{context}
---

User query: {query}

Answer:"""
            else:
                print(f"Warning: No documents retrieved for query: {query[:50]}...")
                prompt = f"""Answer the following user query based on your internal knowledge.

User query: {query}

Answer:"""
            final_prompts.append(prompt)
            
        responses = self.llm.generate(final_prompts)
        return responses

In [6]:
MODEL_NAME = "RAG"

rag = RAG()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

(VllmWorkerProcess pid=195) INFO 04-14 03:02:00 [weight_utils.py:315] No model.safetensors.index.json found in remote.
(VllmWorkerProcess pid=195) INFO 04-14 03:02:04 [loader.py:447] Loading weights took 3.52 seconds
INFO 04-14 03:02:04 [loader.py:447] Loading weights took 3.73 seconds
(VllmWorkerProcess pid=195) INFO 04-14 03:02:04 [model_runner.py:1146] Model loading took 1.6918 GiB and 14.294224 seconds
INFO 04-14 03:02:04 [model_runner.py:1146] Model loading took 1.6901 GiB and 14.382459 seconds
(VllmWorkerProcess pid=195) INFO 04-14 03:02:13 [worker.py:267] Memory profiling takes 8.09 seconds
(VllmWorkerProcess pid=195) INFO 04-14 03:02:13 [worker.py:267] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
(VllmWorkerProcess pid=195) INFO 04-14 03:02:13 [worker.py:267] model weights take 1.69GiB; non_torch_memory takes 0.12GiB; PyTorch activation peak memory takes 0.25GiB; the rest of the memory reserved for KV Cache is 11.20GiB

Capturing CUDA graph shapes:  74%|███████▍  | 26/35 [00:30<00:09,  1.11s/it]

(VllmWorkerProcess pid=195) INFO 04-14 03:02:50 [custom_all_reduce.py:195] Registering 1995 cuda graph addresses


Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:40<00:00,  1.15s/it]

INFO 04-14 03:03:01 [custom_all_reduce.py:195] Registering 1995 cuda graph addresses
(VllmWorkerProcess pid=195) INFO 04-14 03:03:01 [model_runner.py:1598] Graph capturing finished in 41 secs, took 0.21 GiB
INFO 04-14 03:03:01 [model_runner.py:1598] Graph capturing finished in 40 secs, took 0.21 GiB
INFO 04-14 03:03:01 [llm_engine.py:448] init engine (profile, create kv cache, warmup model) took 56.09 seconds


### 1. MedMCQA benchmark

#### Dataset loading and preparing

In [9]:
SEED = 4242
BATCH_SIZE = 4
NUM_SAMPLES = 200
DATASET_MEDMCQA = "openlifescienceai/medmcqa"

In [10]:
ds_medmcqa = load_dataset(DATASET_MEDMCQA, split="validation")
ds_medmcqa = ds_medmcqa.shuffle(seed=SEED).select(range(NUM_SAMPLES))
ds_medmcqa

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/85.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/936k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.48M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
    num_rows: 200
})

In [11]:
ds_medmcqa[0]

{'id': '4653fb7a-ddbf-493b-b4ef-92205582a27a',
 'question': 'Which of the following tooth is not having 5 cusps?',
 'opa': 'Mandibular 2nd Molar',
 'opb': 'Mandibular 1st Molar',
 'opc': 'Mandibular 3rd Molar',
 'opd': 'Maxillary 1st Molar',
 'cop': 0,
 'choice_type': 'single',
 'exp': None,
 'subject_name': 'Dental',
 'topic_name': None}

#### Helper functions definition

In [12]:
def format_prompt_medmcqa(example):
    """Formats a single example into a prompt for the LLM."""
    question = example['question']
    options = {
        "A": example['opa'],
        "B": example['opb'],
        "C": example['opc'],
        "D": example['opd']
    }
    
    prompt = f"""
You are an expert in solving multiple-choice questions accurately and explaining your reasoning clearly.
Given a question and a list of answer choices (A, B, C, D), your task is to:
1. Reason shortly about the question and answer choices to find evidances to support your answer.
2. Identify the correct answer. Please choose the single best answer from the options provided.
3. Output the final answer in the format: Answer: [Option Letter]

Question: {question}
Options:
A. {options['A']}
B. {options['B']}
C. {options['C']}
D. {options['D']}

Reasoning:
    """
    return prompt

In [13]:
def get_ground_truth_medmcqa(example):
    """Maps the correct option index (cop) to the corresponding letter."""
    mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    cop_index = example.get('cop')
    if cop_index is None or cop_index not in mapping:
        print(f"Warning: Invalid 'cop' value found: {cop_index} in example ID {example.get('id')}. Skipping ground truth.")
        return None
    return mapping[cop_index]

In [14]:
def extract_choice_mcq(generated_text):
    """Extracts the predicted choice (A, B, C, or D) from the LLM's output."""
    text = generated_text.strip()

    # Check for phrases like "The answer is A" or "Answer: A"
    match = re.search(r'(?:answer|choice|option) is\s*:?\s*([A-D])', text, re.IGNORECASE)
    if match:
        return match.group(1).upper()

    # Look for the first standalone letter A, B, C, or D in the text
    match = re.search(r'\b([A-D])\b', text)
    if match:
        return match.group(1).upper()

    # Fallback - If no clear choice found, return None
    print(f"Warning: Could not extract answer from text: '{text[:100]}...{text[-100:]}'")
    return None

#### Evaluation

In [15]:
print("\n--- Preparing Prompts and Ground Truths ---")
prompts = [format_prompt_medmcqa(ex) for ex in tqdm(ds_medmcqa, desc="Formatting prompts")]
ground_truths = [get_ground_truth_medmcqa(ex) for ex in tqdm(ds_medmcqa, desc="Extracting ground truths")]
valid_indices = [i for i, gt in enumerate(ground_truths) if gt is not None]

if len(valid_indices) < len(ground_truths):
     print(f"Warning: {len(ground_truths) - len(valid_indices)} examples had invalid ground truths and were excluded.")
     prompts = [prompts[i] for i in valid_indices]
     ground_truths = [ground_truths[i] for i in valid_indices]
     original_indices = valid_indices

if len(prompts) > 0:
    print("\nExample Prompt:")
    print(prompts[0])
    print(f"Corresponding Ground Truth: {ground_truths[0]}")
else:
    print("No valid prompts to evaluate.")
    exit()


--- Preparing Prompts and Ground Truths ---


Extracting ground truths: 100%|██████████| 200/200 [00:00<00:00, 7742.43it/s]


Example Prompt:

You are an expert in solving multiple-choice questions accurately and explaining your reasoning clearly.
Given a question and a list of answer choices (A, B, C, D), your task is to:
1. Reason shortly about the question and answer choices to find evidances to support your answer.
2. Identify the correct answer. Please choose the single best answer from the options provided.
3. Output the final answer in the format: Answer: [Option Letter]

Question: Which of the following tooth is not having 5 cusps?
Options:
A. Mandibular 2nd Molar
B. Mandibular 1st Molar
C. Mandibular 3rd Molar
D. Maxillary 1st Molar

Reasoning:
    
Corresponding Ground Truth: A


In [16]:
print("\n--- Running Inference ---")
all_outputs_text = []
num_batches = math.ceil(len(prompts) / BATCH_SIZE)

for i in tqdm(range(num_batches), desc="Generating Responses"):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, len(prompts))
    batch_prompts = prompts[start_idx:end_idx]
    batch_outputs_text = rag.get_response(batch_prompts, top_k=2)    
    all_outputs_text.extend(batch_outputs_text)

if len(all_outputs_text) > 0:
    print("\nExample Generated Text (raw):")
    print(all_outputs_text[0])


--- Running Inference ---


Generating Responses:   0%|          | 0/50 [00:00<?, ?it/s]

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses:   2%|▏         | 1/50 [00:09<07:36,  9.32s/it]

Processing batch of 4 queries...
Querying DB for 4 queries...


...

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses:  98%|█████████▊| 49/50 [21:36<00:27, 27.73s/it]

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses: 100%|██████████| 50/50 [22:20<00:00, 26.82s/it]


Example Generated Text (raw):
 [Option Letter]

The answer is B. Mandibular 1st Molar. Because Mandibular 1st Molar has only 1 cusp, whereas the others have 2, 3, or 5. Wait, no, the question is asking which is NOT having 5 cusps. So, Mandibular 1st Molar has 1 cusp, so it's not having 5. The others have 2, 3, or 5. So, the correct answer is B.
</think>

Answer: B. Mandibular 1st Molar

The Mandibular 1st Molar has only 1 cusp, which is different from the other options that have 2, 3, or 5 cusps. Therefore, it is the one without 5 cusps.

Answer: B


In [17]:
print("\n--- Extracting Predictions ---")
predictions = [extract_choice_mcq(text) for text in tqdm(all_outputs_text, desc="Extracting choices")]
num_invalid_responces = predictions.count(None)
print(f"\n------------------------------\nNumber of invalid responces: {num_invalid_responces}")

if len(predictions) > 0:
    print("\nExample Extracted Prediction:")
    print(predictions[0])


--- Extracting Predictions ---


Extracting choices: 100%|██████████| 200/200 [00:00<00:00, 6517.25it/s]


The user also provided a previous query and response, which might help in understan...cclusion of the occlusion of the occlusion of the occlusion of the occlusion of the occlusion of the'

The alveolar process is the thinning of the alveolar wall, which occurs in the alve...ther divided into the alveolar space and the alveolar bone. The alveolar bone is located in the alve'

------------------------------
Number of invalid responces: 2

Example Extracted Prediction:
B


In [18]:
print("\n--- Calculating Metrics ---")
correct_count = 0
total_count = len(predictions)
results_by_subject = {}

if total_count != len(ground_truths):
     print(f"Warning: Mismatch between number of predictions ({total_count}) and ground truths ({len(ground_truths)}). This should not happen.")
     total_count = min(total_count, len(ground_truths))

for i in range(total_count):
    original_data_index = original_indices[i] if 'original_indices' in locals() else i
    data_item = ds_medmcqa[original_data_index]
    subject = data_item.get('subject_name', 'Unknown')

    pred = predictions[i]
    truth = ground_truths[i]
    is_correct = (pred == truth)

    if subject not in results_by_subject:
        results_by_subject[subject] = {'correct': 0, 'total': 0}

    if is_correct:
        correct_count += 1
        results_by_subject[subject]['correct'] += 1
    results_by_subject[subject]['total'] += 1

overall_accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0


--- Calculating Metrics ---


In [21]:
print("\n--- Evaluation Results ---")
print(f"Model Evaluated: {MODEL_NAME}")
print(f"Dataset Used: {DATASET_MEDMCQA}")
print(f"Number of Questions Evaluated: {total_count}")
print(f"Number of Correct Answers: {correct_count}")
print(f"Overall Accuracy: {overall_accuracy:.2f}%")

print("\nAccuracy by Subject:")
sorted_subjects = sorted(results_by_subject.keys())
for subject in sorted_subjects:
    counts = results_by_subject[subject]
    sub_acc = (counts['correct'] / counts['total']) * 100 if counts['total'] > 0 else 0
    print(f"- {subject}: {sub_acc:.2f}% ({counts['correct']}/{counts['total']})")


--- Evaluation Results ---
Model Evaluated: RAG
Dataset Used: openlifescienceai/medmcqa
Number of Questions Evaluated: 200
Number of Correct Answers: 58
Overall Accuracy: 29.00%

Accuracy by Subject:
- Anaesthesia: 50.00% (1/2)
- Anatomy: 16.67% (1/6)
- Biochemistry: 12.50% (1/8)
- Dental: 29.85% (20/67)
- ENT: 40.00% (2/5)
- Forensic Medicine: 42.86% (3/7)
- Gynaecology & Obstetrics: 41.18% (7/17)
- Medicine: 33.33% (2/6)
- Microbiology: 50.00% (3/6)
- Ophthalmology: 0.00% (0/4)
- Pathology: 8.33% (1/12)
- Pediatrics: 7.14% (1/14)
- Pharmacology: 33.33% (4/12)
- Physiology: 50.00% (3/6)
- Radiology: 0.00% (0/2)
- Skin: 0.00% (0/1)
- Social & Preventive Medicine: 33.33% (2/6)
- Surgery: 36.84% (7/19)


### 2. MedQA

#### Dataset loading and preparing

In [22]:
SEED = 4242
BATCH_SIZE = 4
NUM_SAMPLES = 200
DATASET_MEDQA = "GBaker/MedQA-USMLE-4-options-hf"
SPLIT_MEDQA = "validation"

In [23]:
ds_medqa = load_dataset(DATASET_MEDQA, split=SPLIT_MEDQA)
ds_medqa = ds_medqa.shuffle(seed=SEED).select(range(NUM_SAMPLES))
ds_medqa

README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

train.json:   0%|          | 0.00/9.77M [00:00<?, ?B/s]

dev.json:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1272 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'sent1', 'sent2', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
    num_rows: 200
})

In [24]:
ds_medqa[1]

{'id': 'dev-00646',
 'sent1': 'A 31-year-old gravida 2 para 2 woman presents to her primary care physician for follow up. Two weeks ago, she gave birth via vaginal delivery to a 9.5 lb (4.3 kg) male infant. The delivery was complicated by a vaginal laceration that required extensive suturing once the infant was delivered. Immediately after delivery of the placenta she experienced intense shaking and chills that resolved within 1 hour. She has felt well since the delivery but admits to 6 days of malodorous smelling vaginal discharge that is tan in color. She has a history of vaginal candidiasis and is worried that it may be recurring. Her temperature is 98.8°F (37.1°C), blood pressure is 122/73 mmHg, pulse is 88/min, respirations are 16/min, and BMI is 33 kg/m^2. Speculum exam reveals a 1.5 cm dark red, velvety lesion on the posterior vaginal wall with a tan discharge. The pH of the discharge is 6.4. Which of the following is the most likely diagnosis?',
 'sent2': '',
 'ending0': 'Bacte

#### Helper functions definition

In [25]:
def format_prompt_medqa(example):
    """Formats a single example into a prompt for the LLM."""
    question = example['sent1']
    options = {
        "A": example['ending0'],
        "B": example['ending1'],
        "C": example['ending2'],
        "D": example['ending3'],
    }
    
    prompt = f"""
You are an expert in solving multiple-choice questions accurately and explaining your reasoning clearly.
Given a question and a list of answer choices (A, B, C, D), your task is to:
1. Reason shortly about the question and answer choices to find evidances to support your answer.
2. Identify the correct answer. Please choose the single best answer from the options provided.
3. Output the final answer in the format: Answer: [Option Letter]

Question: {question}
Options:
A. {options['A']}
B. {options['B']}
C. {options['C']}
D. {options['D']}

Reasoning:
    """
    return prompt

In [26]:
def get_ground_truth_medqa(example):
    """Maps the label to the corresponding letter."""
    mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    label = example.get('label')
    if label is None or label not in mapping:
        print(f"Warning: Invalid 'cop' value found: {label} in example ID {example.get('id')}. Skipping ground truth.")
        return None
    return mapping[label]

#### Evaluation

In [27]:
print("\n--- Preparing Prompts and Ground Truths ---")
prompts = [format_prompt_medqa(ex) for ex in tqdm(ds_medqa, desc="Formatting prompts")]
ground_truths = [get_ground_truth_medqa(ex) for ex in tqdm(ds_medqa, desc="Extracting ground truths")]
valid_indices = [i for i, gt in enumerate(ground_truths) if gt is not None]

if len(valid_indices) < len(ground_truths):
     print(f"Warning: {len(ground_truths) - len(valid_indices)} examples had invalid ground truths and were excluded.")
     prompts = [prompts[i] for i in valid_indices]
     ground_truths = [ground_truths[i] for i in valid_indices]
     original_indices = valid_indices

if len(prompts) > 0:
    print("\nExample Prompt:")
    print(prompts[0])
    print(f"Corresponding Ground Truth: {ground_truths[0]}")
else:
    print("No valid prompts to evaluate.")
    exit()


--- Preparing Prompts and Ground Truths ---


Extracting ground truths: 100%|██████████| 200/200 [00:00<00:00, 9669.31it/s]


Example Prompt:

You are an expert in solving multiple-choice questions accurately and explaining your reasoning clearly.
Given a question and a list of answer choices (A, B, C, D), your task is to:
1. Reason shortly about the question and answer choices to find evidances to support your answer.
2. Identify the correct answer. Please choose the single best answer from the options provided.
3. Output the final answer in the format: Answer: [Option Letter]

Question: A 9-year-old girl is brought to the physician by her father for evaluation of intermittent muscle cramps for the past year and short stature. She has had recurrent upper respiratory tract infections since infancy. She is at the 5th percentile for weight and 10th percentile for height. Physical examination shows nasal polyps and dry skin. An x-ray of the right wrist shows osteopenia with epiphyseal widening. Which of the following sets of laboratory findings is most likely in this patient's serum?
 $$$ Calcium %%% Phosphorus

In [28]:
print("\n--- Running Inference ---")
all_outputs_text = []
num_batches = math.ceil(len(prompts) / BATCH_SIZE)

for i in tqdm(range(num_batches), desc="Generating Responses"):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, len(prompts))
    batch_prompts = prompts[start_idx:end_idx]
    batch_outputs_text = rag.get_response(batch_prompts, top_k=2)
    all_outputs_text.extend(batch_outputs_text)

if len(all_outputs_text) > 0:
    print("\nExample Generated Text (raw):")
    print(all_outputs_text[0])


--- Running Inference ---


Generating Responses:   0%|          | 0/50 [00:00<?, ?it/s]

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses:   2%|▏         | 1/50 [00:44<36:38, 44.87s/it]

Processing batch of 4 queries...
Querying DB for 4 queries...


...

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses:  98%|█████████▊| 49/50 [29:56<00:43, 43.57s/it]

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses: 100%|██████████| 50/50 [30:40<00:00, 36.81s/it]


Example Generated Text (raw):
 A

The patient has a history of recurrent upper respiratory tract infections, which is a common cause of paroxysmal chest pain and muscle cramps. Her short stature and weight suggest a low bone density, which is why the x-ray shows osteopenia. The parathyroid hormone is involved in bone resorption, so its level should be normal. Calcitriol, a parapyrontate, is a calcium supplement and should be normal. However, the calcium level is low, which is typical for a 9-year-old with a history of infections. Parathyroid hormone levels are normal, so the parathyroid hormone level is up. Phosphorus is low because of the short stature. So the findings are: calcium ↓, phosphorus ↓, parathyroid ↑, calcitriol ↓. Wait, but the options don't have this combination. Hmm, maybe I'm missing something.

Wait, the patient has a history of recurrent upper respiratory tract infections, which is a common cause of paroxysmal chest pain and muscle cramps. Her short stature and weig

In [29]:
print("\n--- Extracting Predictions ---")
predictions = [extract_choice_mcq(text) for text in tqdm(all_outputs_text, desc="Extracting choices")]
num_invalid_responces = predictions.count(None)
print(f"\n------------------------------\nNumber of invalid responces: {num_invalid_responces}")

if len(predictions) > 0:
    print("\nExample Extracted Prediction:")
    print(predictions[0])


--- Extracting Predictions ---


Extracting choices: 100%|██████████| 200/200 [00:00<00:00, 4371.28it/s]


The patient is a restrained passenger at the time of impact, which suggests that sh...ant for assessing the heart. The absence of a wrist drop suggests that the shoulder is not a problem'

The patient's jaw episodes are described as intense, shooting pains that occur when...nt’s clinical presentation exits the skull through one of the brain structures.

The options are the'

The patient has recurrent urinary tract infections, which is a common cause of urin...The patient's abdominal examination showing palpable flank masses suggests that the primary issue is'

The patient's symptoms include abnormally weak, decreased appetite, no bowel moveme...s the patient has a decreased potassium level, which is an abnormal test. Alternatively, perhaps the'

The patient is a 70-year-old man with a history of hypertension and Alzheimer's dis...ed with foreign body aspiration or granulomatosis. The patient's symptoms include myalgia, headache,'

The patient presents with respiratory distress, facial

In [30]:
print("\n--- Calculating Metrics ---")
correct_count = 0
total_count = len(predictions)
results_by_subject = {}

if total_count != len(ground_truths):
     print(f"Warning: Mismatch between number of predictions ({total_count}) and ground truths ({len(ground_truths)}). This should not happen.")
     total_count = min(total_count, len(ground_truths))

for i in range(total_count):
    original_data_index = original_indices[i] if 'original_indices' in locals() else i
    data_item = ds_medqa[original_data_index]
    subject = data_item.get('subject_name', 'Unknown')

    pred = predictions[i]
    truth = ground_truths[i]
    is_correct = (pred == truth)

    if subject not in results_by_subject:
        results_by_subject[subject] = {'correct': 0, 'total': 0}

    if is_correct:
        correct_count += 1
        results_by_subject[subject]['correct'] += 1
    results_by_subject[subject]['total'] += 1

overall_accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0


--- Calculating Metrics ---


In [31]:
print("\n--- Evaluation Results ---")
print(f"Model Evaluated: {MODEL_NAME}")
print(f"Dataset Used: {DATASET_MEDQA}")
print(f"Number of Questions Evaluated: {total_count}")
print(f"Number of Correct Answers: {correct_count}")
print(f"Overall Accuracy: {overall_accuracy:.2f}%")


--- Evaluation Results ---
Model Evaluated: RAG
Dataset Used: GBaker/MedQA-USMLE-4-options-hf
Number of Questions Evaluated: 200
Number of Correct Answers: 43
Overall Accuracy: 21.50%


### 3. MMLU medical

#### Dataset loading and preparing

In [32]:
SEED = 4242
BATCH_SIZE = 4
NUM_SAMPLES_SUBSET = 50
NUM_SAMPLES = 200
DATASET_MMLU = "cais/mmlu"
SPLIT_MMLU = "test"

MMLU_MEDICAL_SUBSETS = [
    "anatomy",
    "clinical_knowledge",
    "professional_medicine",
    "college_biology",
    "college_medicine",
    "medical_genetics",
    "professional_medicine"
]

In [33]:
datasets_mmlu = []
for subset in MMLU_MEDICAL_SUBSETS:
    ds = load_dataset(DATASET_MMLU, subset, split=SPLIT_MMLU)
    ds = ds.shuffle(seed=SEED).select(range(NUM_SAMPLES_SUBSET))
    datasets_mmlu.append(ds)


ds_mmlu = concatenate_datasets(datasets_mmlu)
ds_mmlu = ds_mmlu.shuffle(seed=SEED).select(range(NUM_SAMPLES))
ds_mmlu

README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.1k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/5.28k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

...

dev-00000-of-00001.parquet:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'subject', 'choices', 'answer'],
    num_rows: 200
})

In [34]:
ds_mmlu[0]

{'question': 'Mitochondria isolated and placed in a buffered solution with a low pH begin to manufacture ATP. Which of the following is the best explanation for the effect of low external pH?',
 'subject': 'college_biology',
 'choices': ['It increases the concentration of OH-, causing the mitochondria to pump H+ to the intermembrane space.',
  'It increases the OH- concentration in the mitochondria matrix.',
  'It increases the acid concentration in the mitochondria matrix.',
  'It increases diffusion of H+ from the intermembrane space to the matrix.'],
 'answer': 3}

#### Helper functions definition

In [35]:
def format_prompt_mmlu(example):
    """Formats a single example into a prompt for the LLM."""
    question = example['question']
    options = {
        "A": example['choices'][0],
        "B": example['choices'][1],
        "C": example['choices'][2],
        "D": example['choices'][3]
    }
    
    prompt = f"""
You are an expert in solving multiple-choice questions accurately and explaining your reasoning clearly.
Given a question and a list of answer choices (A, B, C, D), your task is to:
1. Reason shortly about the question and answer choices to find evidances to support your answer.
2. Identify the correct answer. Please choose the single best answer from the options provided.
3. Output the final answer in the format: Answer: [Option Letter]

Question: {question}
Options:
A. {options['A']}
B. {options['B']}
C. {options['C']}
D. {options['D']}

Reasoning:
    """
    return prompt

In [36]:
def get_ground_truth_mmlu(example):
    """Maps the label to the corresponding letter."""
    mapping = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    label = example.get('answer')
    if label is None or label not in mapping:
        print(f"Warning: Invalid 'cop' value found: {label} in example ID {example.get('id')}. Skipping ground truth.")
        return None
    return mapping[label]

#### Evaluation

In [37]:
print("\n--- Preparing Prompts and Ground Truths ---")
prompts = [format_prompt_mmlu(ex) for ex in tqdm(ds_mmlu, desc="Formatting prompts")]
ground_truths = [get_ground_truth_mmlu(ex) for ex in tqdm(ds_mmlu, desc="Extracting ground truths")]
valid_indices = [i for i, gt in enumerate(ground_truths) if gt is not None]

if len(valid_indices) < len(ground_truths):
     print(f"Warning: {len(ground_truths) - len(valid_indices)} examples had invalid ground truths and were excluded.")
     prompts = [prompts[i] for i in valid_indices]
     ground_truths = [ground_truths[i] for i in valid_indices]
     original_indices = valid_indices

if len(prompts) > 0:
    print("\nExample Prompt:")
    print(prompts[0])
    print(f"Corresponding Ground Truth: {ground_truths[0]}")
else:
    print("No valid prompts to evaluate.")
    exit()


--- Preparing Prompts and Ground Truths ---


Extracting ground truths: 100%|██████████| 200/200 [00:00<00:00, 11743.98it/s]


Example Prompt:

You are an expert in solving multiple-choice questions accurately and explaining your reasoning clearly.
Given a question and a list of answer choices (A, B, C, D), your task is to:
1. Reason shortly about the question and answer choices to find evidances to support your answer.
2. Identify the correct answer. Please choose the single best answer from the options provided.
3. Output the final answer in the format: Answer: [Option Letter]

Question: Mitochondria isolated and placed in a buffered solution with a low pH begin to manufacture ATP. Which of the following is the best explanation for the effect of low external pH?
Options:
A. It increases the concentration of OH-, causing the mitochondria to pump H+ to the intermembrane space.
B. It increases the OH- concentration in the mitochondria matrix.
C. It increases the acid concentration in the mitochondria matrix.
D. It increases diffusion of H+ from the intermembrane space to the matrix.

Reasoning:
    
Correspond

In [38]:
print("\n--- Running Inference ---")
all_outputs_text = []
num_batches = math.ceil(len(prompts) / BATCH_SIZE)

for i in tqdm(range(num_batches), desc="Generating Responses"):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, len(prompts))
    batch_prompts = prompts[start_idx:end_idx]
    batch_outputs_text = rag.get_response(batch_prompts, top_k=2)
    all_outputs_text.extend(batch_outputs_text)

if len(all_outputs_text) > 0:
    print("\nExample Generated Text (raw):")
    print(all_outputs_text[0])


--- Running Inference ---


Generating Responses:   0%|          | 0/50 [00:00<?, ?it/s]

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses:   2%|▏         | 1/50 [00:44<36:07, 44.24s/it]

Processing batch of 4 queries...
Querying DB for 4 queries...


...

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses:  98%|█████████▊| 49/50 [21:15<00:30, 30.30s/it]

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses: 100%|██████████| 50/50 [21:18<00:00, 25.58s/it]


Example Generated Text (raw):
 [Option Letter]

The answer is D. It increases diffusion of H+ from the intermembrane space to the matrix.

Wait, but I'm a bit confused. Let me think again. Mitochondria are known to produce ATP through the electron transport chain. The process involves a series of proton pumping steps. The mitochondrial matrix is where the electrons are generated, and the intermembrane space is where protons are pumped from the matrix to the matrix (or from the matrix to the intermembrane space? Wait, no, the protons are pumped from the matrix to the intermembrane space, which is then used to generate ATP.

So, if the mitochondrial matrix is in a buffered solution with a low pH, what happens? Low pH means lower H+ concentration. The mitochondrial matrix is rich in protons, so if the pH is low, the concentration of H+ is lower. This would mean that the protons in the intermembrane space are being pumped into the matrix, which is the mitochondrial matrix. Wait, no, the i

In [39]:
print("\n--- Extracting Predictions ---")
predictions = [extract_choice_mcq(text) for text in tqdm(all_outputs_text, desc="Extracting choices")]
num_invalid_responces = predictions.count(None)
print(f"\n------------------------------\nNumber of invalid responces: {num_invalid_responces}")

if len(predictions) > 0:
    print("\nExample Extracted Prediction:")
    print(predictions[0])


--- Extracting Predictions ---


Extracting choices: 100%|██████████| 200/200 [00:00<00:00, 8301.11it/s]


The patient has irregular menses, which is a common condition known as polycystic o...egular menses are a result of the irregular ovulation. The patient's irregular ovulation is a result'

The patient's symptoms and findings are as follows:

1. The patient is a 14-year-ol...r syndrome, hyperprolactinemia, or hyperestrogenism.

83. Benign hyperplasia is a result of hormonal'

The patient has a 31-year-old woman with a history of dyspnea and wheezing, which i...ion. The feral heart rate is 144/min, which is consistent with mitral regurgitation. The feral heart'

The patient is a 17-year-old girl with a stung bee. She's in the emergency departme...hin normal range, so the primary concern is the underlying cause of the wheezing. The correct answer'

The user is asking about the next step in managing this patient. The patient is 18,...e heart sounds are not audible, which is a key finding, so the next step is to perform a chest X-ray'

The patient has a 31-year-old woman with a history of 

In [40]:
print("\n--- Calculating Metrics ---")
correct_count = 0
total_count = len(predictions)
results_by_subject = {}

if total_count != len(ground_truths):
     print(f"Warning: Mismatch between number of predictions ({total_count}) and ground truths ({len(ground_truths)}). This should not happen.")
     total_count = min(total_count, len(ground_truths))

for i in range(total_count):
    original_data_index = original_indices[i] if 'original_indices' in locals() else i
    data_item = ds_mmlu[original_data_index]
    subject = data_item.get('subject', 'Unknown')

    pred = predictions[i]
    truth = ground_truths[i]
    is_correct = (pred == truth)

    if subject not in results_by_subject:
        results_by_subject[subject] = {'correct': 0, 'total': 0}

    if is_correct:
        correct_count += 1
        results_by_subject[subject]['correct'] += 1
    results_by_subject[subject]['total'] += 1

overall_accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0


--- Calculating Metrics ---


In [41]:
print("\n--- Evaluation Results ---")
print(f"Model Evaluated: {MODEL_NAME}")
print(f"Dataset Used: {DATASET_MMLU}")
print(f"Number of Questions Evaluated: {total_count}")
print(f"Number of Correct Answers: {correct_count}")
print(f"Overall Accuracy: {overall_accuracy:.2f}%")

print("\nAccuracy by Subject:")
sorted_subjects = sorted(results_by_subject.keys())
for subject in sorted_subjects:
    counts = results_by_subject[subject]
    sub_acc = (counts['correct'] / counts['total']) * 100 if counts['total'] > 0 else 0
    print(f"- {subject}: {sub_acc:.2f}% ({counts['correct']}/{counts['total']})")


--- Evaluation Results ---
Model Evaluated: RAG
Dataset Used: cais/mmlu
Number of Questions Evaluated: 200
Number of Correct Answers: 61
Overall Accuracy: 30.50%

Accuracy by Subject:
- anatomy: 22.58% (7/31)
- clinical_knowledge: 24.14% (7/29)
- college_biology: 25.93% (7/27)
- college_medicine: 40.00% (12/30)
- medical_genetics: 39.29% (11/28)
- professional_medicine: 30.91% (17/55)


### 4. PubMedQA

#### Dataset loading and preparing

In [42]:
SEED = 4242
BATCH_SIZE = 4
NUM_SAMPLES = 200
DATASET_PUBMEDQA = "qiaojin/PubMedQA"
SUBSET_PUBMEDQA = "pqa_labeled"
SPLIT_PUBMEDQA = "train"

In [43]:
ds_pubmedqa = load_dataset(DATASET_PUBMEDQA, SUBSET_PUBMEDQA, split=SPLIT_PUBMEDQA)
ds_pubmedqa = ds_pubmedqa.shuffle(seed=SEED).select(range(NUM_SAMPLES))
ds_pubmedqa

README.md:   0%|          | 0.00/5.19k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 200
})

In [44]:
ds_pubmedqa[0]

{'pubid': 22504515,
 'question': 'Endovenous laser ablation in the treatment of small saphenous varicose veins: does site of access influence early outcomes?',
 'context': {'contexts': ['The study was performed to evaluate the clinical and technical efficacy of endovenous laser ablation (EVLA) of small saphenous varicosities, particularly in relation to the site of endovenous access.',
   'Totally 59 patients with unilateral saphenopopliteal junction incompetence and small saphenous vein reflux underwent EVLA (810 nm, 14 W diode laser) with ambulatory phlebectomies. Small saphenous vein access was gained at the lowest site of truncal reflux. Patients were divided into 2 groups: access gained above mid-calf (AMC, n = 33) and below mid-calf (BMC, n = 26) levels. Outcomes included Venous Clinical Severity Scores (VCSS), Aberdeen Varicose Vein Questionnaire (AVVQ), patient satisfaction, complications, and recurrence rates.',
   'Both groups demonstrated significant improvement in VCSS, AVV

#### Helper functions definition

In [45]:
def format_prompt_pubmedqa(example):
    """Formats a single example into a prompt for the LLM."""
    question = example['question']
    if not isinstance(example.get('context'), dict) or 'contexts' not in example['context']:
        print(f"Warning: Skipping example due to missing or invalid context field.")
        return None

    context_passages = example['context']['contexts']
    full_context = "\n\n".join(context_passages)

    prompt = f"""
You are an expert in analyzing scientific texts and answering questions based on provided context and explaining your reasoning clearly.
Your task is to determine the answer to the question ('yes', 'no', or 'maybe') based only on the information given in the context. Follow these steps:
1. Analyze the provided context in relation to the question. Summarize the key evidence (or lack thereof) relevant to answering the question. This is your reasoning.
2. Based on your reasoning from the context, determine if the answer to the question is 'yes', 'no', or 'maybe'.
3. Output your reasoning first. After the reasoning, start a new line and provide the final decision in the specific format: Answer: [yes/no/maybe]

Context:
{full_context}

Question: {question}

Reasoning:
    """
    return prompt

In [46]:
def get_ground_truth_pubmedqa(example):
    """Extracts the ground truth ('yes', 'no', 'maybe') from the example."""
    decision = example.get('final_decision')
    if decision not in ['yes', 'no', 'maybe']:
        print(f"Warning: Invalid 'final_decision' value found: {decision}. Skipping ground truth.")
        return None
    return decision

In [47]:
def extract_yes_no_maybe(generated_text):
    """Extracts the predicted choice (yes, no, maybe) from the LLM's output."""
    text = generated_text.strip().lower()

    # Explicit "Answer: yes/no/maybe" potentially followed by punctuation/eos
    match = re.search(r'(?:answer|decision)\s*[:\-]?\s*(yes|no|maybe)\b', text)
    if match:
        return match.group(1)

    # Look for the first occurrence of "yes", "no", or "maybe" as a whole word
    match = re.search(r'\b(yes|no|maybe)\b', text)
    if match:
        return match.group(1)

    # Fallback - If no clear choice found, return None
    print(f"Warning: Could not extract answer from text: '{text[:100]}...{text[-100:]}'")
    return None

#### Evaluation

In [48]:
print("\n--- Preparing Prompts and Ground Truths ---")
prompts = []
ground_truths_raw = []
original_indices_map = []

for i, ex in enumerate(tqdm(ds_pubmedqa, desc="Formatting prompts")):
    prompt = format_prompt_pubmedqa(ex)
    if prompt:
        prompts.append(prompt)
        ground_truths_raw.append(get_ground_truth_pubmedqa(ex))
        original_indices_map.append(i)

valid_indices = [i for i, gt in enumerate(ground_truths_raw) if gt is not None]

if len(valid_indices) < len(prompts):
     invalid_gt_count = len(prompts) - len(valid_indices)
     print(f"Warning: {invalid_gt_count} examples had invalid ground truths and were excluded.")
     prompts = [prompts[i] for i in valid_indices]
     ground_truths = [ground_truths_raw[i] for i in valid_indices]
     original_indices = [original_indices_map[i] for i in valid_indices]
else:
    ground_truths = ground_truths_raw
    original_indices = original_indices_map

if len(prompts) > 0:
    print("\nExample Prompt:")
    print(prompts[0])
    print(f"Corresponding Ground Truth: {ground_truths[0]}")
else:
    print("No valid prompts to evaluate.")
    exit()


--- Preparing Prompts and Ground Truths ---


Formatting prompts: 100%|██████████| 200/200 [00:00<00:00, 6346.16it/s]


Example Prompt:

You are an expert in analyzing scientific texts and answering questions based on provided context and explaining your reasoning clearly.
Your task is to determine the answer to the question ('yes', 'no', or 'maybe') based only on the information given in the context. Follow these steps:
1. Analyze the provided context in relation to the question. Summarize the key evidence (or lack thereof) relevant to answering the question. This is your reasoning.
2. Based on your reasoning from the context, determine if the answer to the question is 'yes', 'no', or 'maybe'.
3. Output your reasoning first. After the reasoning, start a new line and provide the final decision in the specific format: Answer: [yes/no/maybe]

Context:
The study was performed to evaluate the clinical and technical efficacy of endovenous laser ablation (EVLA) of small saphenous varicosities, particularly in relation to the site of endovenous access.

Totally 59 patients with unilateral saphenopopliteal jun

In [49]:
print("\n--- Running Inference ---")
all_outputs_text = []
num_batches = math.ceil(len(prompts) / BATCH_SIZE)

for i in tqdm(range(num_batches), desc="Generating Responses"):
    start_idx = i * BATCH_SIZE
    end_idx = min((i + 1) * BATCH_SIZE, len(prompts))
    batch_prompts = prompts[start_idx:end_idx]
    batch_outputs_text = rag.get_response(batch_prompts, top_k=2)
    all_outputs_text.extend(batch_outputs_text)

if len(all_outputs_text) > 0:
    print("\nExample Generated Text (raw):")
    print(all_outputs_text[0])


--- Running Inference ---


Generating Responses:   0%|          | 0/50 [00:00<?, ?it/s]

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses:   2%|▏         | 1/50 [00:05<04:08,  5.07s/it]

Processing batch of 4 queries...
Querying DB for 4 queries...


...

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses:  98%|█████████▊| 49/50 [24:32<00:18, 18.30s/it]

Processing batch of 4 queries...
Querying DB for 4 queries...



Generating Responses: 100%|██████████| 50/50 [24:35<00:00, 29.51s/it]


Example Generated Text (raw):
 

Answer: 

Now, let's proceed to the step-by-step analysis.
</think>

The study evaluated the efficacy of endovenous laser ablation (EVLA) in small saphenous varicosities, focusing on the site of access. Patients were divided into two groups based on the access level: above mid-calf (AMC, n=33) and below mid-calf (BMC, n=26). Both groups showed significant improvement in clinical and quality of life metrics, with no differences in complications or recurrence rates between the groups. This suggests that the site of access may influence early outcomes.

Answer: Maybe


In [50]:
print("\n--- Extracting Predictions ---")
predictions = [extract_yes_no_maybe(text) for text in tqdm(all_outputs_text, desc="Extracting choices")]
num_invalid_responсes = predictions.count(None)
print(f"\n------------------------------\nNumber of invalid responces: {num_invalid_responсes}")

if len(predictions) > 0:
    print("\nExample Extracted Prediction:")
    print(predictions[0])


--- Extracting Predictions ---


Extracting choices: 100%|██████████| 200/200 [00:00<00:00, 12662.81it/s]


---

pharmacology_katzung. davis mp, walsh d: methadone for r...id poisoning. [14] drug screening is much more useful in screening for occult opioid use in settings'

------------------------------
Number of invalid responces: 1

Example Extracted Prediction:
maybe


In [51]:
print("\n--- Calculating Metrics ---")
correct_count = 0
total_count = len(predictions)
results_by_subject = {}

if total_count != len(ground_truths):
     print(f"Warning: Mismatch between number of predictions ({total_count}) and ground truths ({len(ground_truths)}). This should not happen.")
     total_count = min(total_count, len(ground_truths))

for i in range(total_count):
    original_data_index = original_indices[i] if 'original_indices' in locals() else i
    data_item = ds_pubmedqa[original_data_index]
    subject = data_item.get('subject_name', 'Unknown')

    pred = predictions[i]
    truth = ground_truths[i]
    is_correct = (pred == truth)

    if subject not in results_by_subject:
        results_by_subject[subject] = {'correct': 0, 'total': 0}

    if is_correct:
        correct_count += 1
        results_by_subject[subject]['correct'] += 1
    results_by_subject[subject]['total'] += 1

overall_accuracy = (correct_count / total_count) * 100 if total_count > 0 else 0


--- Calculating Metrics ---


In [52]:
print("\n--- Evaluation Results ---")
print(f"Model Evaluated: {MODEL_NAME}")
print(f"Dataset Used: {DATASET_PUBMEDQA}")
print(f"Number of Questions Evaluated: {total_count}")
print(f"Number of Correct Answers: {correct_count}")
print(f"Overall Accuracy: {overall_accuracy:.2f}%")


--- Evaluation Results ---
Model Evaluated: RAG
Dataset Used: qiaojin/PubMedQA
Number of Questions Evaluated: 200
Number of Correct Answers: 113
Overall Accuracy: 56.50%
